# 日本ユニシス　スキルチェック課題

## 概要
M5 forecastingのコンテストに参加する。売上高で世界最大の企業であるウォルマートの階層的な売上データを用いて、今後28日間の日次売上を予測する。

## 全体の流れ
1. 必要なパッケージのインポート
2. データの前処理
3. EDA
4. モデリング

## 1. 必要なパッケージのインポート

In [ ]:
import os, re
import pandas as pd
import numpy as np
import plotly_express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import seaborn as sns
import gc
import warnings
warnings.filterwarnings('ignore')
from lightgbm import LGBMRegressor
import joblib

## 2. データの前処理
### データのダウンロード
* sales_train_evaluation.csv：製品，店舗ごとの売り上げデータ
* calendar.csv：イベント情報を含んだカレンダー
* sell_prices.csv：店舗ごとの商品販売価格のデータ

In [ ]:
sales = pd.read_csv('/kaggle/input/m5-forecasting-accuracy/sales_train_evaluation.csv')
sales.name = 'sales'
calendar = pd.read_csv('/kaggle/input/m5-forecasting-accuracy/calendar.csv')
calendar.name = 'calendar'
prices = pd.read_csv('/kaggle/input/m5-forecasting-accuracy/sell_prices.csv')
prices.name = 'prices'

In [ ]:
#テストデータの枠をsalesに追加
for d in range(1942,1970):
    col = 'd_' + str(d)
    sales[col] = 0
    sales[col] = sales[col].astype(np.int16)

In [ ]:
#メモリの削減
def downcast(df):
    cols = df.dtypes.index.tolist()
    types = df.dtypes.values.tolist()
    for i,t in enumerate(types):
        if 'int' in str(t):
            if df[cols[i]].min() > np.iinfo(np.int8).min and df[cols[i]].max() < np.iinfo(np.int8).max:
                df[cols[i]] = df[cols[i]].astype(np.int8)
            elif df[cols[i]].min() > np.iinfo(np.int16).min and df[cols[i]].max() < np.iinfo(np.int16).max:
                df[cols[i]] = df[cols[i]].astype(np.int16)
            elif df[cols[i]].min() > np.iinfo(np.int32).min and df[cols[i]].max() < np.iinfo(np.int32).max:
                df[cols[i]] = df[cols[i]].astype(np.int32)
            else:
                df[cols[i]] = df[cols[i]].astype(np.int64)
        elif 'float' in str(t):
            if df[cols[i]].min() > np.finfo(np.float16).min and df[cols[i]].max() < np.finfo(np.float16).max:
                df[cols[i]] = df[cols[i]].astype(np.float16)
            elif df[cols[i]].min() > np.finfo(np.float32).min and df[cols[i]].max() < np.finfo(np.float32).max:
                df[cols[i]] = df[cols[i]].astype(np.float32)
            else:
                df[cols[i]] = df[cols[i]].astype(np.float64)
        elif t == np.object:
            if cols[i] == 'date':
                df[cols[i]] = pd.to_datetime(df[cols[i]], format='%Y-%m-%d')
            else:
                df[cols[i]] = df[cols[i]].astype('category')
    return df  

sales = downcast(sales)
prices = downcast(prices)
calendar = downcast(calendar)

In [ ]:
#売上データをワイドフォーマットからロングフォーマットに変換
df = pd.melt(sales, id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name='d', value_name='sold').dropna()
#データの結合
df = pd.merge(df, calendar, on='d', how='left')
df = pd.merge(df, prices, on=['store_id','item_id','wm_yr_wk'], how='left') 

### データセットの中身を確認
* id：全体のID (型：category，例：HOBBIES_1_001_CA_1_evaluation)
* item_id：アイテムのID (型：category，例：HOBBIES_1_001)
* dept_id：商品部門のID (型：category，例：HOBBIES_1)
* cat_id：商品カテゴリーのID (型：category，例：HOBBIES)
* store_id：店舗のID (型：category，例：CA_1)
* state_id：州のID (型：category，例：CA)
* d：日のID (型：object，例：d_1)
* sold：売り上げ数 (型：int16，例：12)
* date：日付 (型：datetime64[ns]，例：2011-01-29)
* wm_yr_wk：週のID (型：int16，例：11101)
* weekday：曜日 (型：category，例：Saturday)
* wday：曜日のID (型：int8，例：Saturday --> 1, Monday --> 3)
* month：月 (型：int8，例：1)
* year：年 (型：int16，例：2011)
* event_name_1, event_name_2：イベント名 (型：category，例：SuperBowl)
* event_type_1, event_type_2：イベントのタイプ (型：category，例：Cultural)
* snap_CA, snap_TX, snap_WI：SNAPイベントのフラグ (型：int8，例：0 or 1)
* sell_price：商品の値段 (型：float16，例：0.459961)

In [ ]:
print(df.head(20))
print(df.info())
#print(df.tail(20))
#print(pd.unique(df['event_name_1']))
#print(pd.unique(df['event_type_1']))

## 3. EDA
### 季節性
商品の売れ行きには夏や冬などの季節による違い，月曜日や土曜日などの曜日による違いなどが考えられる。
そのため，年単位や月単位，週単位で周期性があるかを確かめる。
* データ分析による考察まとめ
 * 売り上げ数は年々上昇傾向にある。
 * 5月，また11月から年末にかけて落ち込みが見られる
 * 12月25日のクリスマスは全店舗閉店のため，売り上げは0である
 * 土曜日や日曜日の休みの日に売り上げ数は高い。

In [ ]:
#全店舗の売り上げ数の推移
sold=df.groupby('date').sum()['sold']
#30日間の移動平均線
sold_30mean = sold.rolling('30D',center=True).mean()
plt.figure()
plt.plot(sold,c='b') 
plt.plot(sold_30mean,c='r') 
plt.title('all sales')
plt.show()

In [ ]:
#各年ごとの売り上げ数の推移
sold_2012 = sold['20120101':'20121231']
sold_2013 = sold['20130101':'20131231']
sold_2014 = sold['20140101':'20141231']
sold_2015 = sold['20150101':'20151231']
sold_2012_30mean = sold_2012.rolling('30D',center=True).mean()
sold_2013_30mean = sold_2013.rolling('30D',center=True).mean()
sold_2014_30mean = sold_2014.rolling('30D',center=True).mean()
sold_2015_30mean = sold_2015.rolling('30D',center=True).mean()
fig = plt.figure(figsize=(30, 8))
ax1 = fig.add_subplot(141) 
ax1.plot(sold_2012,c='b') 
ax1.plot(sold_2012_30mean,c='r') 
ax1.set_ylim(0, 60000)
ax1.set_title('2012 sales')
ax2 = fig.add_subplot(142) 
ax2.plot(sold_2013,c='b') 
ax2.plot(sold_2013_30mean,c='r') 
ax2.set_ylim(0, 60000)
ax2.set_title('2013 sales')
ax3 = fig.add_subplot(143) 
ax3.plot(sold_2014,c='b') 
ax3.plot(sold_2014_30mean,c='r') 
ax3.set_ylim(0, 60000)
ax3.set_title('2014 sales')
ax4 = fig.add_subplot(144)
ax4.plot(sold_2015,c='b') 
ax4.plot(sold_2015_30mean,c='r') 
ax4.set_ylim(0, 60000)
ax4.set_title('2015 sales')

plt.show()

In [ ]:
#各曜日の総売り上げ
sold_wday = df.groupby('wday').sum()['sold']
weekday = ['Saturday', 'Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
plt.bar(weekday, sold_wday)

### SNAPによる影響
SNAPの影響によって売り上げ数が変化することが考えられる。そのため，SNAPの有無によって変化を確かめる。

#### SNAPについて
SNAP（Supplemental Nutrition Assistance Programは、連邦政府による最大の栄養補助プログラムである。SNAPは、資格のある低所得者とその家族に、電子給付振替カードを介して給付を行う。このカードは、認可された小売食品店で対象となる食品を購入する際に、デビットカードのように使用することができる。

* データ分析による考察まとめ
 * どの州においても売り上げ数はSNAPの日にはSNAPではない日よりも高い

In [ ]:
snap_column_list = ['snap_CA', 'snap_TX', 'snap_WI']
fig, ax = plt.subplots(1, 3, figsize=(30, 8))
for i, snap_column in enumerate(snap_column_list):
    #state名の取得
    state_name = re.sub('snap_', '', snap_column)
    
    #SNAPではない日の州ごとの売り上げ数の表示(売り上げ数：青，30日移動平均：赤)
    df_state = df[df['state_id']==state_name]
    df_nosnap = df_state[df[snap_column]==0]
    sold_nosnap = df_nosnap.groupby('date').sum()['sold']
    sold_nosnap_30mean = sold_nosnap.rolling('30D',center=True).mean()
    ax[i].plot(sold_nosnap, c='b')
    ax[i].plot(sold_nosnap_30mean, c='r')
    
    #SNAPの日の州ごとの売り上げ数(売り上げ数：シアン，30日移動平均：緑)
    df_snap = df_state[df_state[snap_column]==1]
    sold_snap = df_snap.groupby('date').sum()['sold']
    sold_snap_30mean = sold_snap.rolling('30D',center=True).mean()
    ax[i].plot(sold_snap, c='c')
    ax[i].plot(sold_snap_30mean, c='g')
    
    ax[i].set_ylim(0,30000)
    ax[i].set_title(state_name)

## 4.モデリング
### データのクレンジング

In [ ]:
#ユニーク値を辞書型として取得
d_id = dict(zip(df.id.cat.codes, df.id))
d_item_id = dict(zip(df.item_id.cat.codes, df.item_id))
d_dept_id = dict(zip(df.dept_id.cat.codes, df.dept_id))
d_cat_id = dict(zip(df.cat_id.cat.codes, df.cat_id))
d_store_id = dict(zip(df.store_id.cat.codes, df.store_id))
d_state_id = dict(zip(df.state_id.cat.codes, df.state_id))

In [ ]:
#データセット中で何日目かをint型に変換
df.d = df['d'].apply(lambda x: x.split('_')[1]).astype(np.int16)

#カテゴリー変数を整数値にエンコード
cols = df.dtypes.index.tolist()
types = df.dtypes.values.tolist()
for i,type in enumerate(types):
    if type.name == 'category':
        df[cols[i]] = df[cols[i]].cat.codes
        
#意味が重複している日付の削除
df.drop('date',axis=1,inplace=True)

### 特徴量の追加
* ラグ特徴量
 * 週単位や月単位の周期性を考慮し，指定した数日前の特徴量を追加する
* SNAPに関する特徴量(実装できていない)
 * SNAPの日の何日前かと何日後かの特徴量を与える

In [ ]:
#ラグ特徴量の追加(週単位，月単位の周期性を考慮)
lags = [1,2,3,4,5,6,7,14,21,28]
for lag in lags:
    df['sold_lag_'+str(lag)] = df.groupby(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'],as_index=False)['sold'].shift(lag).astype(np.float16)

#ラグ特徴量算出のために発生するNaNの除去
df = df[df['d']>=57]

In [ ]:
# 学習するデータを保存
df.to_pickle('data.pkl')
del df
gc.collect();

### モデルの学習

In [ ]:
#学習データの取得
data = pd.read_pickle('data.pkl')
#validデータとtestデータの取得
valid = data[(data['d']>=1914) & (data['d']<1942)][['id','d','sold']]
test = data[data['d']>=1942][['id','d','sold']]
eval_preds = test['sold']
valid_preds = valid['sold']

In [ ]:
#州ごとのモデルを生成
states = sales.state_id.cat.codes.unique().tolist()
for state in states:
    df = data[data['store_id']==state]
    
    #trainingデータ，validデータ，testデータを分ける
    X_train, y_train = df[df['d']<1914].drop('sold',axis=1), df[df['d']<1914]['sold']
    X_valid, y_valid = df[(df['d']>=1914) & (df['d']<1942)].drop('sold',axis=1), df[(df['d']>=1914) & (df['d']<1942)]['sold']
    X_test = df[df['d']>=1942].drop('sold',axis=1)
    
    #学習
    model = LGBMRegressor(
        n_estimators=1000,
        learning_rate=0.3,
        subsample=0.8,
        colsample_bytree=0.8,
        max_depth=8,
        num_leaves=50,
        min_child_weight=300
    )
    print('*****Prediction for State: {}*****'.format(d_state_id[state]))
    model.fit(X_train, y_train, eval_set=[(X_train,y_train),(X_valid,y_valid)],
             eval_metric='rmse', verbose=20, early_stopping_rounds=20)
    valid_preds[X_valid.index] = model.predict(X_valid)
    eval_preds[X_test.index] = model.predict(X_test)
    filename = 'model'+str(d_state_id[state])+'.pkl'
    # 学習済みモデルの保存
    joblib.dump(model, filename)
    del model, X_train, y_train, X_valid, y_valid
    gc.collect()

### 結果の出力

In [ ]:
#validデータの結果を取得
valid['sold'] = valid_preds
validation = valid[['id','d','sold']]
validation = pd.pivot(validation, index='id', columns='d', values='sold').reset_index()
validation.columns=['id'] + ['F' + str(i + 1) for i in range(28)]
validation.id = validation.id.map(d_id).str.replace('evaluation','validation')

#testデータの結果を取得
test['sold'] = eval_preds
evaluation = test[['id','d','sold']]
evaluation = pd.pivot(evaluation, index='id', columns='d', values='sold').reset_index()
evaluation.columns=['id'] + ['F' + str(i + 1) for i in range(28)]
#Remap the category id to their respective categories
evaluation.id = evaluation.id.map(d_id)

#結果をファイルに出力
submit = pd.concat([validation,evaluation]).reset_index(drop=True)
submit.to_csv('/kaggle/working/submission.csv',index=False)